In [1]:
# Modules
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

# Part 1: Data Scraping and Preparation 

## Step 1: Scraping Competitior's data

In [2]:
# get the spaceWeather data
spaceWeather = requests.get('https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares')
sapceWeather_text = spaceWeather.text
spaceWeather_page = BeautifulSoup(sapceWeather_text)
spaceWeather_page.prettify()[:1000]

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <title>\n   Top 50 solar flares | Solar activity | SpaceWeatherLive.com\n  </title>\n  <meta charset="utf-8"/>\n  <meta content="index, follow, max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots"/>\n  <meta content="On this page you will find an overview of the strongest solar flares since June 1996 together with links to more information in our archive and a v..." name="description"/>\n  <meta content="SpaceWeatherLive, Live, Aurora, Auroral activity, Aurora Australis, Aurora Borealis, northern lights, Solar wind, Kp-index, Space Weather, Space Weather Updates, Aurora forecast, Space Weather Alerts, Solar activity, Solar flares, Sunspots, Aurora alert, Auroral activity, The Sun, SDO, STEREO, EPAM, DSCOVR" name="keywords"/>\n  <!-- Facebook meta -->\n  <meta content="https://spaceweatherlive.com/en/solar-activity/top-50-solar-flares" property="og:url"/>\n  <meta content="article" property="og:type"/>\n  <meta conte

In [3]:
table = spaceWeather_page.find_all('table', class_='table table-striped table-responsive-md')
len(table)

1

In [4]:
# seperate the data into lists
rank = []
x_class = []
date = []
region = []
start_time = []
max_time = []
end_time = []
movie = []

rows = table[0].find_all('tr')
i = 0 
for row in rows:
    if i == 0: # skip the header
        i += 1
        continue
    cols = row.find_all('td')
    rank.append(cols[0].text)
    x_class.append(cols[1].text)
    date.append(cols[2].text)
    region.append(cols[3].text)
    start_time.append(cols[4].text)
    max_time.append(cols[5].text)
    end_time.append(cols[6].text)
    movie.append(cols[7].text)
len(rank), len(x_class), len(date), len(region), len(start_time), len(max_time), len(end_time), len(movie)

(50, 50, 50, 50, 50, 50, 50, 50)

In [5]:
# create a dataframe
spaceWeatherDataFrame = pd.DataFrame({'Rank':rank,'X_class':x_class,'Date':date,'Region':region,'Start_time':start_time,'Max_time':max_time,'End_time':end_time,'Movie':movie})
spaceWeatherDataFrame.head()

,Rank,X_class,Date,Region,Start_time,Max_time,End_time,Movie
0,1,X28+,2003/11/04,0486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,0486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,0808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


## Step 2: Tidy the top 50 solar flare data

In [6]:
# drop the Movie column
modifiedDataFrame=spaceWeatherDataFrame
modifiedDataFrame.drop(['Movie'],axis=1,inplace=True)
modifiedDataFrame.head()

,Rank,X_class,Date,Region,Start_time,Max_time,End_time
0,1,X28+,2003/11/04,0486,19:29,19:53,20:06
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03
2,3,X17.2+,2003/10/28,0486,09:51,11:10,11:24
3,4,X17+,2005/09/07,0808,17:17,17:40,18:03
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55


### Adding the date to Start_time, End_time, Max_time columns 
- Compining the string values of the **Date** column and each value of the **time** columns
- Dropping the **Date** column

In [7]:
modifiedDataFrame['Start_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['Start_time']
modifiedDataFrame['End_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['End_time']
modifiedDataFrame['Max_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['Max_time']
modifiedDataFrame.drop(['Date'], axis=1, inplace=True)
modifiedDataFrame.head()

,Rank,X_class,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003/11/04 19:29,2003/11/04 19:53,2003/11/04 20:06
1,2,X20+,9393,2001/04/02 21:32,2001/04/02 21:51,2001/04/02 22:03
2,3,X17.2+,0486,2003/10/28 09:51,2003/10/28 11:10,2003/10/28 11:24
3,4,X17+,0808,2005/09/07 17:17,2005/09/07 17:40,2005/09/07 18:03
4,5,X14.4,9415,2001/04/15 13:19,2001/04/15 13:50,2001/04/15 13:55


### Converting the Time columns from String to Pandas TimeStamp
- using `pd.to_datetime`

In [8]:
modifiedDataFrame['Start_time'] = pd.to_datetime(modifiedDataFrame['Start_time'])
modifiedDataFrame['End_time'] = pd.to_datetime(modifiedDataFrame['End_time'])
modifiedDataFrame['Max_time'] = pd.to_datetime(modifiedDataFrame['Max_time'])
modifiedDataFrame[:10]

,Rank,X_class,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00
5,6,X10,0486,2003-10-29 20:37:00,2003-10-29 20:49:00,2003-10-29 21:01:00
6,7,X9.4,8100,1997-11-06 11:49:00,1997-11-06 11:55:00,1997-11-06 12:01:00
7,8,X9.3,2673,2017-09-06 11:53:00,2017-09-06 12:02:00,2017-09-06 12:10:00
8,9,X9,0930,2006-12-05 10:18:00,2006-12-05 10:35:00,2006-12-05 10:45:00
9,10,X8.3,0486,2003-11-02 17:03:00,2003-11-02 17:25:00,2003-11-02 17:39:00


In [9]:
modifiedDataFrame.dtypes

Rank                  object
X_class               object
Region                object
Start_time    datetime64[ns]
Max_time      datetime64[ns]
End_time      datetime64[ns]
dtype: object

In [10]:
# check for missing values
for col in modifiedDataFrame.columns[:3]:
    print(modifiedDataFrame[col].str.contains('-').sum())

0
0
0


In [11]:
# save the SpaceWeather dataset
modifiedDataFrame.to_csv('space.csv')

## Step3: Scrape the NASA data

In [12]:
# get the website content
reb = requests.get("http://www.hcbravo.org/IntroDataSci/misc/waves_type2.html")
reb.text[:1000]

'<html><body>\n<h2>Wind/WAVES type II bursts and CMEs</h2>\n<a href="waves_type2_description.htm">A Brief Description</a>\n<pre>\nNOTE: List includes DH type II bursts observed by Wind spacecraft, \nbut after STEREO launch on Oct 2006 the start and end times and \nfrequencies of bursts are determined using both Wind and STEREO \nobservations\n\n=================================================================================================\n                DH Type II                       Flare                     CME                   \n----------------------------------------   -----------------   --------------------------   Plots\nStart            End          Frequency     Loc   NOAA  Imp    Date  Time CPA  Width  Spd        \n(1)        (2)   (3)   (4)   (5)    (6)     (7)    (8)  (9)    (10)  (11) (12)  (13) (14)   (15) \n=================================================================================================\n1997/04/01 14:00 04/01 14:15  <a href="http://cdaw.gsfc.nas

In [13]:
page=BeautifulSoup(reb.text)
page.prettify()[:1000]

'<html>\n <body>\n  <h2>\n   Wind/WAVES type II bursts and CMEs\n  </h2>\n  <a href="waves_type2_description.htm">\n   A Brief Description\n  </a>\n  <pre>\nNOTE: List includes DH type II bursts observed by Wind spacecraft, \nbut after STEREO launch on Oct 2006 the start and end times and \nfrequencies of bursts are determined using both Wind and STEREO \nobservations\n\n=================================================================================================\n                DH Type II                       Flare                     CME                   \n----------------------------------------   -----------------   --------------------------   Plots\nStart            End          Frequency     Loc   NOAA  Imp    Date  Time CPA  Width  Spd        \n(1)        (2)   (3)   (4)   (5)    (6)     (7)    (8)  (9)    (10)  (11) (12)  (13) (14)   (15) \n=================================================================================================\n1997/04/01 14:00 04/01 14:15  <a

### Using Regex to scrap the content of the NASA website
- getting the rows of the website divided into lists of values
- e.g. `['1997/04/01', '14:00', '04/01', '14:15', '8000', '4000', 'S25E16', '8026', 'M1.3', '04/01', '15:18', '74', '79', '312', 'PHTX']`

In [14]:
rex=r'\d+/.*'
l=re.findall(rex,page.text)
newList=[]
for i in l:
    newList.append(i.split())
newList[0]

['1997/04/01',
 '14:00',
 '04/01',
 '14:15',
 '8000',
 '4000',
 'S25E16',
 '8026',
 'M1.3',
 '04/01',
 '15:18',
 '74',
 '79',
 '312',
 'PHTX']

In [15]:
# put the values into seperated lists
startDate=[]
startTime=[]
endDate=[]
endTime=[]
startFrequency=[]
endFrequency=[]
flare_Location=[]
flare_region=[]
importance=[]
CME_Date=[]
CME_Time=[]
CPA=[]
width = []
speed = []
for i in newList:
    startDate.append(i[0])
    startTime.append(i[1])
    endDate.append(i[2])
    endTime.append(i[3])
    startFrequency.append(i[4])
    endFrequency.append(i[5])
    flare_Location.append(i[6])
    flare_region.append(i[7])
    importance.append(i[8])
    CME_Date.append(i[9])
    CME_Time.append(i[10])
    CPA.append(i[11])
    width.append(i[12])
    speed.append(i[13])

In [16]:
# put the lists into Pandas df
df = pd.DataFrame({'startDate':startDate,'startTime':startTime,'endDate':endDate,
                   'endTime':endTime,'startFrequency':startFrequency,'endFrequency':endFrequency,
                   'flare_Location': flare_Location,'flare_region':flare_region,'importance':importance,
                   'CME_Date':CME_Date,'CME_Time':CME_Time, 'width':width, 'speed':speed, 'CPA':CPA})
df.head()

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,79,312,74
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,360,878,Halo
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,360,464,Halo
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,165,296,263
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,155,712,133


In [17]:
len(df.columns)

14

In [18]:
df.columns

Index(['startDate', 'startTime', 'endDate', 'endTime', 'startFrequency',
       'endFrequency', 'flare_Location', 'flare_region', 'importance',
       'CME_Date', 'CME_Time', 'width', 'speed', 'CPA'],
      dtype='object')

### Missing Values per Column
- missing values: has `-` in it 

In [19]:
missing_cols = []
for col in df.columns:
    s = df[col].str.contains('-').sum()
    print(col,':', s)
    if s>0:
        missing_cols.append(col)

startDate : 0
startTime : 0
endDate : 0
endTime : 0
startFrequency : 0
endFrequency : 0
flare_Location : 6
flare_region : 83
importance : 105
CME_Date : 20
CME_Time : 20
width : 20
speed : 20
CPA : 21


In [20]:
# columns with missing values
missing_cols

['flare_Location',
 'flare_region',
 'importance',
 'CME_Date',
 'CME_Time',
 'width',
 'speed',
 'CPA']

In [21]:
# discovering rows with missing data
df[df[col].str.contains('-')]

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
96,2000/11/09,16:15,11/11,04:00,10000,40,S11E10,9221,M1.0,--/--,--:--,----,----,----
149,2001/10/09,11:20,10/09,11:55,14000,2000,S28E08,9653,----,--/--,--:--,----,----,----
150,2001/10/09,13:10,10/09,23:00,5000,50,BACK,-----,----,--/--,--:--,----,----,----
229,2003/07/10,14:10,07/10,16:10,4000,400,N13W90,10397,M3.6,--/--,--:--,----,----,----
250,2004/01/05,03:40,01/05,03:50,9000,2500,S12E38,10536,M6.9,--/--,--:--,----,----,----
261,2004/06/22,22:07,06/22,22:30,10000,7000,S12W24,10635,C1.7,--/--,--:--,----,----,----
262,2004/06/23,06:30,06/23,08:55,14000,5000,S09W21,10635,C2.5,--/--,--:--,----,----,----
269,2004/09/19,17:15,09/19,18:15,14000,2500,N03W58,10672,M1.9,--/--,--:--,----,----,----
284,2005/01/04,11:20,01/04,11:35,13000,6000,N05W11,10715,C7.3,--/--,--:--,----,----,----
298,2005/06/16,20:25,06/16,21:40,9000,1000,N08W90,10775,M4.0,--/--,--:--,----,----,----


## Step 4: Tidy the NASA dataframe

In [22]:
# get a copy of the data before proceding 
new_df = df.copy()

In [23]:
# before replacing missing values
new_df.CPA[95:100]

95     271
96    ----
97     257
98    Halo
99     124
Name: CPA, dtype: object

In [24]:
# replace all missing valus with nan in CPA column
new_df.loc[new_df['CPA'].str.contains('-'), 'CPA'] = 'nan'

In [25]:
# after replacing missing with nan
new_df.CPA[95:100]

95     271
96     nan
97     257
98    Halo
99     124
Name: CPA, dtype: object

### Replace any missing entries as `nan` in all columns

In [26]:
# replace missing with nan
for col in df.columns:
    new_df.loc[new_df[col].str.contains('-'), col] = 'nan'

In [27]:
new_df[90:140]

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
90,2000/09/25,02:20,09/25,03:00,14000,1000,N09W18,9169,M1.8,09/25,02:50,360,587,Halo
91,2000/10/05,22:10,10/05,22:35,5000,1250,N25W90,9177,nan,10/05,18:26,86,1046,323
92,2000/10/16,07:10,10/16,08:00,14000,1000,N03W90,9182,M2.5,10/16,07:27,360,1336,Halo
93,2000/10/25,09:30,10/25,24:00,10000,300,N09W63,9199,C4.0,10/25,08:26,360,770,Halo
94,2000/11/03,18:35,11/03,18:45,4000,2500,N02W02,9213,C3.2,11/03,18:26,360,291,Halo
95,2000/11/08,23:20,11/09,12:00,4000,200,N10W77,9213,M7.4,11/08,23:06,>170,1738,271
96,2000/11/09,16:15,11/11,04:00,10000,40,S11E10,9221,M1.0,nan,nan,nan,nan,nan
97,2000/11/12,14:25,11/12,14:40,3000,2000,S14E05,9227,C4.4,11/12,14:50,50,581,257
98,2000/11/23,08:16,11/23,08:35,14000,9000,S26W40,9238,C5.4,11/23,06:06,360,492,Halo
99,2000/11/23,21:00,11/23,21:06,3500,3000,S20E60,9239,C7.9,11/23,21:30,148,1198,124


### is_halo Column
1. If **CPA** row contains *Halo* then **isHalo** is set to `True`
2. Replace *Halo* in **CPA** column with `na`

In [28]:
# new column to indicate halo
new_df['is_halo'] = new_df['CPA'].eq('Halo')


# replace halo with na
new_df.loc[new_df['CPA'].str.contains('Halo'), 'CPA'] = 'na'

In [29]:
new_df[['CPA', 'is_halo']][:10]

,CPA,is_halo
0,74,False
1,na,True
2,na,True
3,263,False
4,133,False
5,240,False
6,233,False
7,na,True
8,na,True
9,98,False


### Creating **lower_bound** column
- if **width** row contains `>` then **lower_bound** is set to `True`
- Remove `>` from the column
- Remove all non-numerical character
 - found `h` then remove it

In [30]:
# Lower bound column
new_df['lower_bound'] = new_df.width.str.contains('>')

In [31]:
# remove non-numberic characters
new_df.loc[new_df.width.str.contains('>'), 'width'] = new_df.loc[new_df.width.str.contains('>'), 'width'].replace(r'>', '', regex=True)
new_df.loc[new_df.width.str.contains('h'), 'width'] = new_df.loc[new_df.width.str.contains('h'), 'width'].replace(r'h', '', regex=True)

In [32]:
new_df.width.str.contains('h').sum()

0

In [33]:
new_df.width.str.contains('>').sum()

0

In [34]:
new_df[:30]

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA,is_halo,lower_bound
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,79,312,74,False,False
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,360,878,na,True,False
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,360,464,na,True,False
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,165,296,263,False,False
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,155,712,133,False,False
5,1997/11/03,05:15,11/03,12:00,14000,250,S20W13,8100,C8.6,11/03,05:28,109,227,240,False,False
6,1997/11/03,10:30,11/03,11:30,14000,5000,S16W21,8100,M4.2,11/03,11:11,122,352,233,False,False
7,1997/11/04,06:00,11/05,04:30,14000,100,S14W33,8100,X2.1,11/04,06:10,360,785,na,True,False
8,1997/11/06,12:20,11/07,08:30,14000,100,S18W63,8100,X9.4,11/06,12:10,360,1556,na,True,False
9,1997/11/27,13:30,11/27,14:00,14000,7000,N17E63,8113,X2.6,11/27,13:56,91,441,98,False,False


### Compine Date and Time columns for Start, End and CME
- endDate: getting the years value from **startDate** column before compining it with **endTime** column
- startDate: compining **startDate** with **startTime** columns

In [35]:
new_df['endDate'] = new_df['startDate'].str.slice(start=0, stop=5) + new_df['endDate'] + ' ' + new_df['endTime']
new_df['startDate'] = new_df['startDate'] + ' ' + new_df['startTime']

# drop endTime and startTime columns
new_df.drop(['endTime', 'startTime'], axis=1, inplace=True)

# rename
new_df.rename(columns={'startDate': 'Start_Datetime', 'endDate':'End_Datetime'}, inplace=True)

In [36]:
new_df.dtypes

Start_Datetime    object
End_Datetime      object
startFrequency    object
endFrequency      object
flare_Location    object
flare_region      object
importance        object
CME_Date          object
CME_Time          object
width             object
speed             object
CPA               object
is_halo             bool
lower_bound         bool
dtype: object

## Custom Function for datetime conversion
- some time data had `24` hours instead of `0` in it so we made a special case for it 
 - By setting the `hours` to `0` and incrementing days by `1`

In [37]:
def custom_to_datetime(date):
    # If the time is 24, set it to 0 and increment day by 1
    if date[11:13] == '24':
        return pd.to_datetime(date[:-5]) + pd.Timedelta(days=1)
    else:
        return pd.to_datetime(date)

In [38]:
# applying custom_to_datetime function to time columns
for col in ['Start_Datetime', 'End_Datetime']:
    new_df[col] = new_df[col].apply(custom_to_datetime)

In [39]:
new_df.dtypes

Start_Datetime    datetime64[ns]
End_Datetime      datetime64[ns]
startFrequency            object
endFrequency              object
flare_Location            object
flare_region              object
importance                object
CME_Date                  object
CME_Time                  object
width                     object
speed                     object
CPA                       object
is_halo                     bool
lower_bound                 bool
dtype: object

In [40]:
# Saving the nasa dataset
new_df.to_csv('nasa.csv')